# Lab 5: LLM by API

# 1. LLM API

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()  # 敏感信息不能明文展示
api_key = os.getenv('OPENAI_API_KEY')

## 1.1 基础API

In [ ]:
from openai import OpenAI

client = OpenAI(
            api_key=api_key,
            base_url="https://api.aaaapi.com/v1"
        )
# 创建一个简单的聊天完成请求
user_prompt = "你好,请介绍一下你自己"

response = client.chat.completions.create(
    model="gpt-4.1",  # gemini-2.5-flash-preview-05-20, gpt-4o (注意.)
    messages=[
        {"role": "user", "content": user_prompt}
    ]
)


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {response.choices[0].message.content}")


## side: Langchain

Langchain是一个封装后的工具。对于工程化有兴趣的同学可以去研究。某些Agent包也会使用Langchain的接口

In [ ]:
%pip install -qU langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=api_key,
    base_url="https://api.aaaapi.com/v1"
)

print(f"用户: {user_prompt}")
print(f"AI助手: {llm.invoke(user_prompt).content}")

## 1.2 调用推理模型

学习使用[官方tutorial](https://platform.openai.com/docs/guides/reasoning?api-mode=chat)

In [ ]:
prompt = """
今有雉兔同笼，上有三十五头，下有九十四足，问鸡兔各几何
"""
response = client.chat.completions.create(
    model="o1-mini", # 推荐测试用 o1-mini, 实际使用o3-mini,
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ],
    
)

assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {prompt}")
print(f"AI助手: {assistant_reply}")

In [ ]:
response.choices[0]

系统提示词会改变整个LLM的应答行为。在Agent体系中，Agent的角色逻辑会被写到System Prompt里

In [ ]:
system_prompt = "你是一个谜语人，总是用谜语来对话。谜底必须对应用户的答案"
response = client.chat.completions.create(
    model="gpt-4.1",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt}
    ]
)
# 返回助手的回复
assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {assistant_reply}")

增加对话上下文

In [ ]:
response = client.chat.completions.create(
    model="gpt-4.1",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "我是一本无字之书，读来似懂非懂。无需打开，便在手中展现玄机。这书名是啥？"},
        {"role": "user", "content": "难道你是天书？"}
    ]
)

print(f"用户: 难道你是天书？")
print(f"AI助手: {response.choices[0].message.content}")

提问，当我们说Prompt Engineering时，我们说的是什么？
- A: System Prompt
- B: 1st User Prompt
- C: All User Prompt
- D: Chat History?

# 2. 多模态大模型

## 2.1 图像理解

In [ ]:
# Display the image
file_path = "data/test_img.png"
from IPython.display import Image
Image(filename=file_path)

In [32]:
import base64
# Read and encode the image
local_img_payload = "data:image/png;base64," + base64.b64encode(open(file_path, "rb").read()).decode('utf-8')

In [ ]:

# Create vision completion
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": "Why is this comic funny? Please explain the humor in chinese"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": local_img_payload
                    }
                }
            ]
        }
    ],
    max_tokens=300  # 控制输出长度
)

print(response.choices[0].message.content)

## 2.2 图像生成

学习使用[官方文档](https://platform.openai.com/docs/guides/images-vision?api-mode=chat)

In [35]:
import base64

img = client.images.generate(
    model="gpt-image-1",
    prompt="A cute baby sea otter",
    n=1,
    size="1024x1024"
)

image_bytes = base64.b64decode(img.data[0].b64_json)



In [ ]:
# 先从内存中显示生成的图片
from IPython.display import Image, display

# 直接从内存中的字节数据显示
display(Image(data=image_bytes))

print("图片已从内存显示完成！")


In [ ]:
with open("output.png", "wb") as f:
    f.write(image_bytes)

print("图片已保存到 output.png 文件！")

## 2.3 如何让cursor读取外部信息

点击"@web"启用搜索功能（不是很好用）

点击"@"符号，增加docs url。很多库有api docs

# 3. Now You Try!

让GPT帮你生成一个故事的4个分镜，然后调用gpt-image-1生成4张图。

**NOTE**
- 如果你想独立做agent项目，不要使用Cursor自动生成代码
- HINT：使用json mode 来确定分镜的数据结构，然后循环调用 images接口

## 挑战题

让cursor帮你设计一个以静态网页存储的绘本。绘本里有图片和文字。